In [25]:
using JuMP
using Flux
using Flux: logitcrossentropy, normalise, onecold, onehotbatch, glorot_uniform, @functor
using Statistics: mean
using Zygote
using LinearAlgebra

## Simple Nueral Net for classification on Iris dataset

In [2]:
# Initialize hyperparameter arguments
struct Args
    lr::Float64
    repeat::Int
    
    Args() = new(0.5, 110)
end

function get_processed_data(args)
    labels = Flux.Data.Iris.labels()
    features = Flux.Data.Iris.features()

    # Subract mean, divide by std dev for normed mean of 0 and std dev of 1.
    normed_features = normalise(features, dims=2)

    klasses = sort(unique(labels))
    onehot_labels = onehotbatch(labels, klasses)

    # Split into training and test sets, 2/3 for training, 1/3 for test.
    train_indices = [1:3:150 ; 2:3:150]

    X_train = normed_features[:, train_indices]
    y_train = onehot_labels[:, train_indices]

    X_test = normed_features[:, 3:3:150]
    y_test = onehot_labels[:, 3:3:150]

    #repeat the data `args.repeat` times
    train_data = Iterators.repeated((X_train, y_train), args.repeat)
    test_data = (X_test,y_test)

    return train_data, test_data
end

# Accuracy Function
accuracy(x, y, model) = mean(onecold(model(x)) .== onecold(y))

# Testing model performance on test data 
function test(model, test)
    X_test, y_test = test
    accuracy_score = accuracy(X_test, y_test, model)
    println("Accuracy: $accuracy_score")
end

accuracy (generic function with 1 method)

In [17]:
function train()
    args = Args()
    
    #Loading processed data
    train_data, test_data = get_processed_data(args)

    model = Chain(Dense(4,10), Dense(10,3), X -> relu.(X))

    # Defining loss function to be used in training
    # For numerical stability, we use here logitcrossentropy
    loss(x, y) = logitcrossentropy(model(x), y)

    # Training
    # Gradient descent optimiser with learning rate `args.lr`
    optimiser = Descent(args.lr)

    println("Starting training.")
    Flux.train!(loss, params(model), train_data, optimiser)

    return model, test_data
end

model, test_data = train()
test(model, test_data)

Starting training.
Accuracy: 0.94


## NN using relu defined by QPTH

In [ ]:
function train()
    args = Args()
    
    #Loading processed data
    train_data, test_data = get_processed_data(args)
    
    # Using a Relu layer defined by qpth
    # y = f(x) = max(0,x) 
     
    # minimize  1/2||x − y||^2
    # s.t.      0 ≤ y 
    
    # dont pass matrices, pass a model instead, that way you can expand later

    model = Chain(Dense(4,10), Dense(10,3), QPTH(,Matrix(I*-1.0,3,3),zeros(3)))

    # Defining loss function to be used in training
    # For numerical stability, we use here logitcrossentropy
    loss(x, y) = logitcrossentropy(model(x), y)

    # Training
    # Gradient descent optimiser with learning rate `args.lr`
    optimiser = Descent(args.lr)

    println("Starting training.")
    Flux.train!(loss, params(model), train_data, optimiser)

    return model, test_data
end

model, test_data = train()
test(model, test_data)

## Define a QP layer

In [22]:
struct QPTH
    Q::Array{Float64}
    p::Array{Float64}
    G::Array{Float64}
    h::Array{Float64}
    A::Array{Float64}
    b::Array{Float64}
    
    QPTH(Q_::Array{Float64},p_::Array{Float64},G_::Array{Float64},h_::Array{Float64}) = new(Q_,p_,G_,h_,zeros(0),zeros(0))
end

In [3]:
# turn it into a callable
@functor QPTH

function (qp::QPTH)(x::AbstractArray)
    Q_,p_,G_,h_,A_,b_ = qp.Q_,p_,G_,h_
    W, b = a.W, a.b
    (W*x .+ b)
    # here should be the forward pass
    # TODO: Use qpth logic here
end

In [17]:
# TODO: Custom adjoint not defined but not reflected in Flux training
Zygote.@adjoint QPLayer(a, b) = QPLayer(a, b), qp -> (qp.b, qp.W)

In [33]:
Matrix(I*-1.0,3,3)

3×3 Array{Float64,2}:
 -1.0   0.0   0.0
  0.0  -1.0   0.0
  0.0   0.0  -1.0

In [34]:
zeros(3)

3-element Array{Float64,1}:
 0.0
 0.0
 0.0